In [48]:
import spacy
from spacy import displacy

import re
import pandas as pd
from textacy import extract

from collections import defaultdict 
from fuzzywuzzy import fuzz
import time
import uuid


In [8]:
dir(time)
print(time.localtime())

time.struct_time(tm_year=2023, tm_mon=11, tm_mday=25, tm_hour=11, tm_min=20, tm_sec=55, tm_wday=5, tm_yday=329, tm_isdst=0)


In [9]:
#files
pcode_file = "D://projects//_external_files//cod_files//combined_locations//locations.csv"
situation_reports = "c://temp//100_situation_reports.csv"

## Load geolocation_services


In [10]:
df_location = pd.read_csv(pcode_file)

def get_pcode_from_location(loc, country_prefix='XX', lang_code='all'):

    if country_prefix != 'XX': #if the country prefix is set, limit search to that
        df_loc = df_location[df_location['pcode_prefix'] == country_prefix]
    else:
        df_loc = df_location

    if lang_code != 'all': #secondary filter - especially important to remove dupes with diff langs share the same script
        df_loc = df_loc[df_loc['lang_code'] == lang_code]
        
    matches = df_loc['pcode'][df_loc['location_name'].str.lower() == loc.lower()].tolist()

    #if the match fails, try again on the normalized name
    if len(matches) == 0:
        #remove common variations in names that can cause misses
        n_loc = re.sub(r'[^a-zA-Z]', '', loc)

        #this will cause problems for non-English.. so if then len is 0, exit
        if len(n_loc) == 0:
            return []
            
        matches = df_loc['pcode'][df_loc['location_normalized'].str.lower() == n_loc.lower()].tolist()
        

    #now check results
    if len(matches) > 1:
        #print(f"more than 1 matches... likely due to different granularity of entities with the same name (ie. Herat City in Herat Province) {matches}")
        #print(f"returning the lowest granularity match. {min(matches, key=len)}")
        #print("if the pcodes are all the same granularity.... you get the first element.")
        return min(matches, key=len)
            
        return matches
    elif len(matches) == 1:
        return matches

    else:
        #couldn't find a match, do a fuzzy search
        compare_list = list(set(df_loc['location_name'].tolist()))
        possible_matches=[]
        for i in compare_list:
            if fuzz.ratio(loc,i) > 70:
                possible_matches.append(i)
                print (f"No exact match to '{loc}'. see if these alternative spellings are correct: {possible_matches}")

    
    return []

assert get_pcode_from_location('istanbul')[0] == 'TUR034'

def get_adm_lvl_from_pcode(pcode):
    return list(set(df_location['adm_lvl'][df_location['pcode'] == pcode].tolist()))
    
def get_name_in_lang(pcode, lang='en'):
    return list(set(df_location['location_name'][(df_location['pcode'] == pcode) & (df_location['lang_code'] == lang)].tolist()))

def get_descendents_of(pcode, lang='en', include_self=True):
    if include_self==True:
        return df_location[df_location['pcode'].str.contains(pcode) & (df_location['lang_code'] == lang)]
    else:
        return df_location[df_location['pcode'].str.contains(pcode) & (df_location['lang_code'] == lang)\
        & (df_location['pcode'] != pcode)]

def get_admin_chain(pcode, lang='en'):
    split_pcode = df_location['split_pcode'][df_location['pcode'] == pcode].tolist()[0]
    levels = split_pcode.split(".")
    pc =''
    admin_chain = []
    #rebuild the pcode one level at a time
    for i in levels:
        pc = pc + i
        admin_chain.append(df_location['location_name'][(df_location['pcode'] == pc) & (df_location['lang_code'] == lang)].tolist()[0])

    return admin_chain

def get_all_locations(lang_code='all'):

    #return all unique location names
    if lang_code == 'all':
        return list(set(df_location['location_name'].to_list()))
    else:
        return list(set(df_location['location_name'][df_location['lang_code'] == lang_code].to_list()))




In [11]:



#ruler = nlp.add_pipe('entity_ruler', before='ner')
#ruler.add_patterns(skills)

## Load Preprocessing Routines

In [12]:
def string_preprocess(text):
    
    def convert_spelled_nums_to_digit(token):
        mappings = {
            'one' : 1,'two' : 2,'three' : 3,'four' : 4,'five' : 5,'six' : 6,'seven' : 7,'eight' : 8,'nine' : 9, 'ten' : 10
            ,'eleven' : 11, 'twelve' : 12, 'thirteen':13, 'fourteen':14, 'fifteen':15, 'sixteen':16, 'seventeen':17
            ,'eighteen':18, 'nineteen':19, 'twenty':20
        }
    
        if mappings.get(token) is not None:
            return mappings[token]
        else:
            return token
        
    text = text.replace("\n"," ")
    text = text.replace("-","_") #replace so tokenization doesn't separate

    #turn 'four' into 4
    text = ' '.join([str(convert_spelled_nums_to_digit(t)) for t in text.split(" ")])


    # remove content in parentheses
    #processed_string = re.sub(r'\([^)]*\)', '', input_string)

    #remove all non alpha numeric and punctuation
    pattern = r'[^a-zA-Z0-9\s\,\.\?\!\-\(\)]'
    text = re.sub(pattern, '', text)

    pattern = r'(\d+)\s+million'
    text = re.sub(r'(\d+)\s+million', r'\1,000,000', text)

    
    #remove commas that serve as thousands separators
    #Hack... fix this so I don't have to run it 3x
    text = re.sub(r'(\d+),(\d+)', r'\1\2', text)
    text = re.sub(r'(\d+),(\d+)', r'\1\2', text)
    text = re.sub(r'(\d+),(\d+)', r'\1\2', text)
    text = text.replace("\s+","\s")
    return text

def string_remove_parenthetical_content(text):
    # Use regular expression to remove content inside parentheses
    text = re.sub(r'\([^)]*\)', '', text)
    return text
    




## Load NLP routines

In [13]:
nlp = spacy.load("en_core_web_md")

# Create patterns and add to the entity ruler to better find locations

all_locs = get_all_locations(lang_code='en')
gpes = []

STOP_LOCS = ['of','can']
all_locs = [e for e in all_locs if e.lower() not in STOP_LOCS]

# create pattern rules for locations based on the COD files
for l in all_locs:
    token_sequence=[]
    for token in l.split('\s+'):
        token_sequence.append({"LOWER":token.lower()})
    x = {'label':'GPE', 'pattern': token_sequence, 'id':get_pcode_from_location(l, lang_code='en')[0]}
    gpes.append(x)
    #print(get_pcode_from_location(l, lang_code='en'))

ruler = nlp.add_pipe('entity_ruler', before='ner')
ruler.add_patterns(gpes)

In [14]:
#keyword_indicators
indicators = {
    'i_people' : ['people','person','child','man','woman']
    ,'i_civilian' : ['civilian']
    ,'i_killed' : ['dead','fatal','die','kill','deceased'] #think about how to incorporate 2 co-existing terms "648 people who lost their lives"
    ,'i_injured' : ['injure','wound']
    ,'i_damage' : ['damage','destroy','collapse']
    ,'i_health_infrastructure' : ['hospital','surgery']
    ,'i_education_infrastructure' : ['school','university']
    ,'i_cash_xfer' : ['xx']
    ,'i_wash' : ['sanitation','water','sewer','drain','drainage']
    ,'i_shelter' : ['shelter','tent','camp','blanket']
    ,'i_food' : ['food','cook','stove','feed','feed','nutrient','meal']
    ,'i_health' : ['health','medical','medicine']
    ,'i_gender_vuln' : ['dignity','gender','pregnant','lactate','lactating']
    ,'i_protection' : ['trauma','mental']
    ,'i_response_capacity' : ['personnel']
    ,'i_other_infrastructure' : ['communicate','radio','internet','telecommunication','electric','line']
    ,'i_money' : ['grant','loan','finance','appeal','chf','fund']
    ,'i_other' : ['biometric']
    ,'i_problem' : ['challenge']
    ,'i_demand_side' : ['need','demand','gap','priority', 'receive'] # note receive implies both supply and demand
    ,'i_supply_side' : ['response','contribute','provide','source','address','deploy','receive'] # note receive implies both supply and demand

    ,'i_assessments' : ['assess','assessment']
}

In [15]:
def get_future_tense_verb(doc):
    def is_future_tense(token):
        #Check if a token is indicative of future tense.
        return (
            token.tag_ == "MD" and token.text.lower() == "will"
            or (token.dep_ == "aux" and token.head.lemma_ == "will")
        )

    for t in doc:
        if is_future_tense(t):
            return f"{t.text} {t.head}"

def find_and_add_indicator(df, indicators):
    ind_counter = []
    for ind in indicators:
  
        df[ind] = df['lower_lemmas'].apply(lambda x: 1 if len([w for w in x if w in indicators[ind]])>0 else 0)
        ind_counter.append(ind)
        #print(ind_counter)
    df['i_count'] = df[ind_counter].sum(axis=1)

    return df


def declare_primary_record_type(row):

    if row['i_count'] == 0:
        return 'background'
    elif row['i_supply_side']:
        return 'response_details'
    elif row['i_demand_side']:
        return 'demand_side'
    elif row[['i_damage','i_health_infrastructure','i_education_infrastructure']].sum() > 0:
        return 'damage_to_homes_and_infrastructure'
    else:
        return 'other'




def obtain_killed_numeric_value(doc):

    key_values = []
    just_count = []
    
    def check_flags(lst):
        for l in lst:
            if l == -1:
                return False
        return True

    #doc = doc.tolist()[0]
    attribute = -1
    noun = -1
    count = -1

    for t in doc:
        if str(t).isdigit():
            count = t
        if t in indicators['i_people']:
            noun = t
        if t in indicators['i_killed']:
            attribute = t

        if check_flags([noun,attribute,count]):

            noun_att_cnt = (noun,attribute,count)
            key_values.append(noun_att_cnt)
            just_count.append(count)

            noun = -1
            attribute = -1
            count = -1

    #changing to return only the count
    return just_count
    #return key_values
            
    
def obtain_injured_numeric_value(doc):

    key_values = []
    just_count = []
    
    def check_flags(lst):
        for l in lst:
            if l == -1:
                return False
        return True

    #doc = doc.tolist()[0]
    attribute = -1
    noun = -1
    count = -1

    for t in doc:
        if str(t).isdigit():
            count = t
        if t in indicators['i_people']:
            noun = t
        if t in indicators['i_injured']:
            attribute = t

        if check_flags([noun,attribute,count]):

            noun_att_cnt = (noun,attribute,count)
            key_values.append(noun_att_cnt)
            just_count.append(count)

            noun = -1
            attribute = -1
            count = -1

    #changing to return only the count
    return just_count
    #return key_values

def obtain_counted_noun_chunks(doc):
    counted_things = []
    for x in list(extract.noun_chunks(doc)):
        for token in x:
            if str(token).isdigit():
                counted_things.append(x)
                continue
    if len(counted_things) > 0:
        return counted_things
    else:
        return ''



def obtain_all_entities(doc):

    STOP_ENTS = ['WASH','PSS','GTC','PFA','NFI','IYCF']
    #stop_ents = STOP_ENTS
    ents = list(extract.entities(doc))
    if len(ents) < 1:
        return None
    entities = defaultdict(list) 
    for e in ents:
        #if e.text not in stop_ents:
        entities[e.label_].append(e)

    return entities   

def extract_entities(row):
    entities = row['entities']
    if entities is None:
        return ''
    en=[]
    for label in entities:
        for e in entities.get(label):
            ent = ' '.join([w.text for w in e]).strip()
            en.append([label,ent])
            
    return en


def extract_ncs(row):
    #data type, list of spans
    xs = row['noun_chunks']
    if xs is None:
        return ''
    en=[]

    for e in xs:
        ent = ' '.join([w.text for w in e]).strip()
        en.append(['NOUN_CHUNK',ent])
    return en


def extract_numeric_key_values(row):
    #data type, list of spans
    xs = row['num_others']
    if xs is None:
        return ''
    return_list=[]

    for e in xs:
        prefix = ''
        numeric = ''
        suffix = ''

        for token in e:
            if token.is_alpha == False:
                numeric = token.text
            elif numeric == '': #alpha but numeric not set yet, this is prefix
                prefix = prefix + ' ' + token.text
            else:
                suffix = suffix + ' ' + token.text
        
        return_list.append([prefix.strip(),numeric,suffix.strip()])   
        
    return return_list


def split_key_value_in_df(field,delim=','):

    s = pd.Series({'prefix' : field, 'left_label' : field, 'right_label' : field})
    
    if isinstance(field, list):
        fields = field
    elif isinstance(field, str):
        fields = field.split(delim)
    else:
        print(field)
    
     
    if len(fields) == 2:
        s = pd.Series({'prefix' : '', 'left_label' : fields[0], 'right_label' : fields[1]})
    elif len(fields) == 3:
        s = pd.Series({'prefix' : fields[0], 'left_label' : fields[1], 'right_label' : fields[2]})



    return s

def split_key_value_in_df_orig(field,left_label="d",right_label="f",delim=','):

    s = pd.Series({left_label : field, right_label : field})
    
    if isinstance(field, list):
     
        if len(field) == 2:
            s = pd.Series({left_label : field[0], right_label : field[1]})

    elif isinstance(field, str):
        fields = field.split(delim)
        if len(fields) == 2:
            s = pd.Series({left_label : fields[0], right_label : fields[1]})

    return s

## Now build base DF

In [28]:
df = pd.read_csv(situation_reports)
df['string_sentence'] = df['text'].apply(lambda x: string_preprocess(x).split('.'))
df = df.explode('string_sentence')
df['string_sentence'] = df['string_sentence'].apply(lambda x: x.strip() + '.')
df = df.reset_index(drop=True)
df

,record_type,source_url,glide_id,source_level_country,source_title,source_desc,source_original_text,reference_url,text,authoring_org,reported_date,string_sentence
0,situation report,https://api.reliefweb.int/v1/reports/4017691,NaN,World,Multi-country outbreak of mpox (monkeypox) - E...,health,**Highlights**,https://reliefweb.int/attachments/2e5a83c9-d6f...,**Highlights**,WHO,2023-11-25T00:00:00+00:00,Highlights.
1,situation report,https://api.reliefweb.int/v1/reports/4017691,NaN,World,Multi-country outbreak of mpox (monkeypox) - E...,health,- The mpox surveillance reporting frequency ha...,https://reliefweb.int/attachments/2e5a83c9-d6f...,- The mpox surveillance reporting frequency ha...,WHO,2023-11-25T00:00:00+00:00,The mpox surveillance reporting frequency has ...
2,situation report,https://api.reliefweb.int/v1/reports/4017691,NaN,World,Multi-country outbreak of mpox (monkeypox) - E...,health,- The mpox surveillance reporting frequency ha...,https://reliefweb.int/attachments/2e5a83c9-d6f...,- The mpox surveillance reporting frequency ha...,WHO,2023-11-25T00:00:00+00:00,.
3,situation report,https://api.reliefweb.int/v1/reports/4017691,NaN,World,Multi-country outbreak of mpox (monkeypox) - E...,health,- A total of 668 new laboratory-confirmed case...,https://reliefweb.int/attachments/2e5a83c9-d6f...,- A total of 668 new laboratory-confirmed case...,WHO,2023-11-25T00:00:00+00:00,A total of 668 new laboratoryconfirmed cases w...
4,situation report,https://api.reliefweb.int/v1/reports/4017691,NaN,World,Multi-country outbreak of mpox (monkeypox) - E...,health,- A total of 668 new laboratory-confirmed case...,https://reliefweb.int/attachments/2e5a83c9-d6f...,- A total of 668 new laboratory-confirmed case...,WHO,2023-11-25T00:00:00+00:00,"The most affected regions, ordered by number o..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3351,situation report,https://api.reliefweb.int/v1/reports/4016821,EP-2023-000181-SDN,South Sudan,South Sudan: Response to the Sudan Crisis Situ...,contributions; coordination; food and nutritio...,"- Currently, the onward transportation assista...",https://reliefweb.int/attachments/a94aa91b-f5d...,"- Currently, the onward transportation assista...",OCHA,2023-11-22T00:00:00+00:00,"As a result, there are a backlog of people in ..."
3352,situation report,https://api.reliefweb.int/v1/reports/4016821,EP-2023-000181-SDN,South Sudan,South Sudan: Response to the Sudan Crisis Situ...,contributions; coordination; food and nutritio...,"- Currently, the onward transportation assista...",https://reliefweb.int/attachments/a94aa91b-f5d...,"- Currently, the onward transportation assista...",OCHA,2023-11-22T00:00:00+00:00,.
3353,situation report,https://api.reliefweb.int/v1/reports/4016821,EP-2023-000181-SDN,South Sudan,South Sudan: Response to the Sudan Crisis Situ...,contributions; coordination; food and nutritio...,- The number of Sudanese refugees and asylum s...,https://reliefweb.int/attachments/a94aa91b-f5d...,- The number of Sudanese refugees and asylum s...,OCHA,2023-11-22T00:00:00+00:00,The number of Sudanese refugees and asylum see...
3354,situation report,https://api.reliefweb.int/v1/reports/4016821,EP-2023-000181-SDN,South Sudan,South Sudan: Response to the Sudan Crisis Situ...,contributions; coordination; food and nutritio...,- The number of Sudanese refugees and asylum s...,https://reliefweb.int/attachments/a94aa91b-f5d...,- The number of Sudanese refugees and asylum s...,OCHA,2023-11-22T00:00:00+00:00,.


In [29]:
idx = df.sample(1).index[0]
print(df.loc[idx]['source_original_text']) #.tolist()[0]
print()
print(df.loc[idx]['string_sentence']) #.tolist()[0]


La ville de Bamako a hébergé l’atelier national du cycle de programmation humanitaire (HPC) pour l’année 2024. Cet exercice annuel a réuni les représentants des services étatiques, les acteurs humanitaires et du développement, et les donateurs autour de l’analyse conjointe des besoins humanitaires des populations vulnérables, ainsi que les réponses à apporter pour sauver des vies. Cet atelier, qui s’est tenu sur deux jours, a permis de définir les populations, les zones, ainsi que les secteurs d’interventions prioritaires du prochain Plan de Réponse Humanitaire de l’année 2024.  

Cet exercice annuel a runi les reprsentants des services tatiques, les acteurs humanitaires et du dveloppement, et les donateurs autour de lanalyse conjointe des besoins humanitaires des populations vulnrables, ainsi que les rponses  apporter pour sauver des vies.


In [34]:
#b build out initial dataframe
df['spacy_doc'] = df['string_sentence'].apply(lambda x: nlp(x))
df['lower_lemmas'] = df['spacy_doc'].apply(lambda x: [w.lemma_.lower() for w in x])

df['string_sent_wo_parens'] = df['string_sentence'].apply(string_remove_parenthetical_content)
df['spacy_wo_parens'] = df['string_sent_wo_parens'].apply(lambda x: nlp(x))
df['wo_parens_lower_lemmas'] = df['spacy_wo_parens'].apply(lambda x: [w.lemma_.lower() for w in x])
df['locations'] = df['spacy_doc'].apply(lambda doc: [e.text for e in doc.ents if e.label_ == 'GPE'])
df['dates'] = df['spacy_doc'].apply(lambda doc: [e.text for e in doc.ents if e.label_ == 'DATE'])
df['svot'] = df['spacy_wo_parens'].apply(lambda doc: list(extract.subject_verb_object_triples(doc)))
df['future_verbs'] = df['spacy_doc'].apply(get_future_tense_verb)

    


In [39]:
df = find_and_add_indicator(df, indicators)
df['record_type'] = df.apply(declare_primary_record_type, axis=1)

In [40]:
df['num_killed'] = df['wo_parens_lower_lemmas'][df['i_killed'] == 1].apply(obtain_killed_numeric_value)
df['num_injured'] = df['wo_parens_lower_lemmas'][df['i_injured'] == 1].apply(obtain_injured_numeric_value)
df['num_others'] = df['spacy_wo_parens'].apply(obtain_counted_noun_chunks)
df['noun_chunks'] = df['spacy_wo_parens'].apply(lambda doc: list(extract.noun_chunks(doc)))
df['entities'] = df['spacy_wo_parens'].apply(obtain_all_entities)

In [61]:
def generate_uuid(x):
    foo = uuid.uuid4().hex
    return foo
    
df['sent_idx'] = df['sent_idx'].apply(generate_uuid)

In [62]:
df['sent_idx']

0       2f6a3718d850403dbb154f55c3ddd5f8
1       77aa2570faf34a7aa28cf188f8aa1612
2       90eea03650b44d5cae260937c6ebf76f
3       2cdb73e9830142498ed9821a7381453c
4       30026648414a4de7a02f0a0631279dfa
                      ...               
3351    bf46d33314734662bd5099c8e0f0174d
3352    326faa0ad06c4f68ba36fb5bdf3ac07e
3353    dcdbbebe724a4b25b6a7ed9fb6245b04
3354    3d360ea807664028b217b55c30795649
3355    8826eaa450aa4c8294fee6c8379c8887
Name: sent_idx, Length: 3356, dtype: object

In [19]:
df.to_csv("c://temp//foo.csv")

In [63]:
#use this as a repeatable-ish pattern for expanding on all the qualitative fields
df_entities = df[['source_url','sent_idx','string_sentence','entities']][df['entities'].isna() == False].copy()
df_entities['tmp'] = df_entities.apply(extract_entities, axis=1)
df_entities = df_entities.drop(columns=['entities'])
df_entities = df_entities.explode('tmp')
df_entities['rec_type'] = 'ENTITY'
df_entities[['rec_prefix','rec_key','rec_value']] = df_entities.apply(lambda x: split_key_value_in_df(x.tmp), axis=1)

#now noun_chunks
df_nouns = df[['source_url','sent_idx','string_sentence','noun_chunks']][df['noun_chunks'].isna() == False].copy()
df_nouns['tmp'] = df_nouns.apply(extract_ncs, axis=1)
df_nouns = df_nouns.drop(columns=['noun_chunks'])
df_nouns = df_nouns.explode('tmp')
df_nouns['rec_type'] = 'NOUN_SEQUENCE'
df_nouns = df_nouns[df_nouns['tmp'].isna() == False].copy()
df_nouns[['rec_prefix','rec_key','rec_value']] = df_nouns.apply(lambda x: split_key_value_in_df(x.tmp), axis=1)

#quantitative values
df_quants = df[['source_url','sent_idx','string_sentence','num_others']][df['num_others'] != ''].copy()
df_quants['tmp'] = df_quants.apply(extract_numeric_key_values, axis=1)
df_quants = df_quants.drop(columns=['num_others'])
df_quants = df_quants.explode('tmp')
df_quants['rec_type'] = 'QUANTIFIED_NOUN'
df_quants[['rec_prefix','rec_key','rec_value']] = df_quants.apply(lambda x: split_key_value_in_df(x.tmp), axis=1)


In [64]:
df_attributes = pd.concat([df_quants, df_nouns,df_entities])
df_attributes = df_attributes.drop(columns=['tmp'])
df_attributes


,source_url,sent_idx,string_sentence,rec_type,rec_prefix,rec_key,rec_value
3,https://api.reliefweb.int/v1/reports/4017691,2cdb73e9830142498ed9821a7381453c,A total of 668 new laboratoryconfirmed cases w...,QUANTIFIED_NOUN,,668,new laboratoryconfirmed cases
3,https://api.reliefweb.int/v1/reports/4017691,2cdb73e9830142498ed9821a7381453c,A total of 668 new laboratoryconfirmed cases w...,QUANTIFIED_NOUN,,29,countries
6,https://api.reliefweb.int/v1/reports/4017691,671d84cf04e24b0e93f922a2cb305646,Eight laboratoryconfirmed cases were reported ...,QUANTIFIED_NOUN,,1,case
11,https://api.reliefweb.int/v1/reports/4017691,cff07493847140c1960b49d1fc7bf0ec,This summary highlights the reporting since 1 ...,QUANTIFIED_NOUN,,1,January
19,https://api.reliefweb.int/v1/reports/4017686,e267fabf2f8c4d6a88b2b5771d246358,A total of 8287 individuals have fled Jilib an...,QUANTIFIED_NOUN,,8287,individuals
...,...,...,...,...,...,...,...
3353,https://api.reliefweb.int/v1/reports/4016821,dcdbbebe724a4b25b6a7ed9fb6245b04,The number of Sudanese refugees and asylum see...,ENTITY,,PERSON,WunthowJoda
3353,https://api.reliefweb.int/v1/reports/4016821,dcdbbebe724a4b25b6a7ed9fb6245b04,The number of Sudanese refugees and asylum see...,ENTITY,,DATE,previous weeks
3355,https://api.reliefweb.int/v1/reports/4016821,8826eaa450aa4c8294fee6c8379c8887,Disruption of supply lines to Abyei Administra...,ENTITY,,ORG,Abyei Administrative Area
3355,https://api.reliefweb.int/v1/reports/4016821,8826eaa450aa4c8294fee6c8379c8887,Disruption of supply lines to Abyei Administra...,ENTITY,,GPE,Sudan


In [65]:
#Join base and attribute df
df_joined = df.merge(df_attributes[['sent_idx','rec_type','rec_prefix','rec_key','rec_value']], left_on='sent_idx', right_on='sent_idx', how='left').copy()
df_joined.explode('locations')
df_joined['locations'] = df_joined['locations'].apply(lambda x: x[0] if len(x)==1 else '')
df_joined.explode('dates')
df_joined['dates'] = df_joined['dates'].apply(lambda x: x[0] if len(x)==1 else '')


df_joined.explode('svot')
df_joined.shape

(21322, 54)